In [2]:
number_of_baskets = 10000000 #Modify to generate a smaller or larger test dataset as preferred

In [3]:
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)

In [ ]:
import pyspark.sql.functions as f
df = sqlContext.range(0, number_of_baskets)
# Generate basket identifiers 
df.withColumn('basket', f.regexp_extract(f.concat(f.lit('0000000000'), df.id), ".*([A-Za-z0-9]{10}$)", 1))